In [1]:
model_id = "google/flan-t5-base"

In [2]:
!pip install pytesseract datasets rouge-score nltk tensorboard py7zr --upgrade
! pip install -U accelerate
! pip install -U transformers
#!apt install git-lfs

!sudo apt-get install git-lfs --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.6 MB/s eta 0:0

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.htm

In [ ]:
#transformers.__version__

In [5]:
import accelerate
accelerate.__version__

'0.22.0'

In [6]:
pip install evaluate Pillow==9.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [7]:
import locale
locale.getpreferredencoding()

'UTF-8'

In [8]:
import transformers
import datasets
import evaluate
from datasets import load_dataset , load_from_disk
from evaluate import load

dataset = load_dataset("billsum",split='train[16000:]')
dataset = dataset.train_test_split(test_size=0.2)
print(dataset)
dataset_id="billsum"
metric = load("rouge")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 2359
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 590
    })
})


In [9]:
dataset = dataset.shuffle()

In [10]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
'''from datasets import concatenate_datasets
#padding
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=["text", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["text", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")'''

'from datasets import concatenate_datasets\n#padding\ntokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=["text", "summary"])\nmax_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])\nprint(f"Max source length: {max_source_length}")\n\ntokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["text", "summary"])\nmax_target_length = max([len(x) for x in tokenized_targets["input_ids"]])\nprint(f"Max target length: {max_target_length}")'

In [11]:
prefix = "summarize: "
input_max_length = 1024
output_max_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=input_max_length, truncation=True)
    labels = tokenizer(
        text_target=examples["summary"], max_length=output_max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#preprocessing function
#from transformers import AutoTokenizer
#from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import GenerationConfig
'''def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary", "title"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")'''

'def preprocess_function(sample,padding="max_length"):\n    # add prefix to the input for t5\n    inputs = ["summarize: " + item for item in sample["text"]]\n\n    # tokenize inputs\n    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)\n\n    # Tokenize targets with the `text_target` keyword argument\n    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)\n\n    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore\n    # padding in the loss.\n    if padding == "max_length":\n        labels["input_ids"] = [\n            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]\n        ]\n\n    model_inputs["labels"] = labels["input_ids"]\n    return model_inputs\n\ntokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary", "title"])\nprint(f"Keys

In [12]:
tokenized_dataset = dataset.map( preprocess_function, batched=True, remove_columns=["title", "text", "summary"])


Map:   0%|          | 0/2359 [00:00<?, ? examples/s]

Map:   0%|          | 0/590 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [14]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100 ,4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


In [16]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [17]:
'''#import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}'''

'#import nltk\nimport numpy as np\n\ndef compute_metrics(eval_pred):\n    predictions, labels = eval_pred\n    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)\n    # Replace -100 in the labels as we can\'t decode them.\n    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)\n    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)\n\n    # Rouge expects a newline after each sentence\n    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]\n    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]\n\n    # Note that other metrics may not have a `use_aggregator` parameter\n    # and thus will return a list, computing a metric for each sentence.\n    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)\n    # Extract a few results\n    result = {key: value * 100 for key, value in result.i

In [18]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=True, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=8,
    # logging & evaluation strategies
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    #metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

In [19]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
2,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
3,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
4,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
5,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
6,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
7,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017
8,0.000000,nan,14.041000,6.012000,11.306800,12.055100,16.061017


TrainOutput(global_step=18872, training_loss=0.0, metrics={'train_runtime': 6237.0744, 'train_samples_per_second': 3.026, 'train_steps_per_second': 3.026, 'total_flos': 2.5740799230541824e+16, 'train_loss': 0.0, 'epoch': 8.0})

In [22]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_rouge1': 14.041,
 'eval_rouge2': 6.012,
 'eval_rougeL': 11.3068,
 'eval_rougeLsum': 12.0551,
 'eval_gen_len': 16.061016949152542,
 'eval_runtime': 160.5961,
 'eval_samples_per_second': 3.674,
 'eval_steps_per_second': 0.461,
 'epoch': 8.0}

In [23]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

'https://huggingface.co/VS18/flan-t5-base-billsum/tree/main/'

In [24]:
from transformers import pipeline
from random import randrange

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="VS18/flan-t5-base-billsum", device=0)
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['text']}\n---------------")
res = summarizer(sample["text"])
print(f"flan-t5-small summary:\n{res[0]['summary_text']}")

Token indices sequence length is longer than the specified maximum sequence length for this model (2537 > 512). Running this sequence through the model will result in indexing errors


dialogue: 
SECTION 1. SHORT TITLE.

    This Act may be cited as the ``Fair Trade Opportunities Act''.

SEC. 2. STATEMENT OF PURPOSE.

    It is the purpose of this Act to enable the President to--
            (1) remedy the actions or policies of countries that do not 
        accord adequate trade benefits to the United States, including 
        substantially equal competitive opportunities for the commerce 
        of the United States;
            (2) negotiate more equitable tariff and other trade 
        benefits for the people of the United States;
            (3) sanction non market economy countries that deny or 
        unduly restrict the right or opportunity of their citizens to 
        emigrate; and
            (4) adjust and simplify the trade laws of the United States 
        to better respond to the realities of a post-Cold War world.

SEC. 3. REPEAL OF JACKSON-VANIK.

    (a) Title IV of Trade Act of 1974.--Title IV of the Trade Act of 
1974 (19 U.S.C. 2431 and fol